In [ ]:
# Câu 3: Thay đổi RGB + ngẫu nhiên biến đổi ảnh
import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as iio
from PIL import Image
import math
import os
import random

# Thư mục chứa ảnh
folder = "exercise"
output_folder = "output_cau3"
os.makedirs(output_folder, exist_ok=True)

def image_inverse(img):
    return 255 - img

def gamma_correction(img, gamma=2.2):
    norm = img / 255.0
    corrected = np.power(norm, gamma)
    return (corrected * 255).astype(np.uint8)

def log_transform(img):
    c = 255 / math.log(1 + np.max(img))
    return (c * np.log(1 + img)).astype(np.uint8)

def histogram_equalization(img):
    hist, bins = np.histogram(img.flatten(), bins=256, range=[0,256])
    cdf = hist.cumsum()
    cdf_normalized = cdf * 255 / cdf[-1]
    img_equalized = np.interp(img.flatten(), bins[:-1], cdf_normalized)
    return img_equalized.reshape(img.shape).astype(np.uint8)

def contrast_stretching(img):
    a, b = np.min(img), np.max(img)
    stretched = (img - a) * 255 / (b - a)
    return stretched.astype(np.uint8)

transformations = [
    ('I', image_inverse),
    ('G', gamma_correction),
    ('L', log_transform),
    ('H', histogram_equalization),
    ('C', contrast_stretching)
]

def list_images(folder):
    images = []
    for file in os.listdir(folder):
        if file.lower().endswith(('.jpg', '.png')):
            images.append(f"{folder}/{file}")
    return images

filenames = list_images(folder)
for path in filenames:
    filename = path.split("/")[-1]
    img = Image.open(path).convert('RGB')
    img_np = np.array(img)

    # Hoán vị RGB ngẫu nhiên
    channels = [0, 1, 2]
    random.shuffle(channels)
    img_shuffled = img_np[:, :, channels]

    # Chuyển sang grayscale để biến đổi
    img_gray = Image.fromarray(img_shuffled).convert('L')
    img_gray_np = np.array(img_gray)

    # Chọn biến đổi ngẫu nhiên từ danh sách
    label, func = random.choice(transformations)
    transformed = func(img_gray_np)

    plt.imshow(transformed, cmap='gray')
    plt.title(f"{filename} - RGB shuffled + {label}")
    plt.axis('off')
    plt.show()

    out_path = f"{output_folder}/{filename.split('.')[0]}_RGB_{label}.png"
    iio.imwrite(out_path, transformed)
